In [72]:
# Import all necessary modules
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score

In [5]:
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: c:\Users\gonza\Documents\VSCode Repo\Datathon\files


# Read all databases

In [8]:
# Reading all the files
nrows = 10000
# balances = pd.read_csv("files/balances.csv", nrows=nrows)
# customers = pd.read_csv("files/customers.csv", nrows=nrows)
# digital = pd.read_csv("files/digital.csv")
# liabilities = pd.read_csv("files/liabilities.csv", nrows=nrows)
# movements = pd.read_csv("files/movements.csv", nrows=nrows)
# sample_submission = pd.read_csv("files/sample_submission.csv", nrows=nrows)
# universe_test = pd.read_csv("files/universe_test.csv")
# universe_train = pd.read_csv("files/universe_train.csv")


# # Reading all the files
# nrows = 10000
balances = pd.read_csv("balances.csv")
customers = pd.read_csv("customers.csv")
digital = pd.read_csv("digital.csv")
liabilities = pd.read_csv("liabilities.csv")
movements = pd.read_csv("movements.csv")
sample_submission = pd.read_csv("sample_submission.csv")
universe_test = pd.read_csv("universe_test.csv")
universe_train = pd.read_csv("universe_train.csv")

# Merge and Aggregate balances

In [9]:
# Merge universe_test with balances
merged_train = pd.merge(left = universe_train, right = balances, on=['ID'], how='left', suffixes=['_universe,', '_balances'])

In [55]:
# Dummy encode 'product' and 'entity' columns during aggregation
dummy_product = pd.get_dummies(merged_train['product'], prefix='product')
dummy_entity = pd.get_dummies(merged_train['entity'], prefix='entity')
dummy_type = pd.get_dummies(merged_train['type'], prefix='type')
merged_train_dummy = pd.concat([merged_train[['ID', 'month', 'balance_amount', 'days_default']], dummy_product, dummy_entity, dummy_type], axis=1)

balance_agg = merged_train_dummy.groupby(['ID']).agg({
    # Create aggregations as specified
    'month': 'count',
    'balance_amount': [len, sum, min, max, np.median, np.std],
    'days_default': [len, sum, min, max, np.median, np.std],
    # Dummification
    # Select columns with a specific prefix (e.g., 'product_' or 'entity_') and sum them
    **{col: 'sum' for col in merged_train_dummy.head().columns if col.startswith('product_')},
    **{col: 'sum' for col in merged_train_dummy.head().columns if col.startswith('entity_')},
    **{col: 'sum' for col in merged_train_dummy.head().columns if col.startswith('type_')}
}).reset_index()

# Display the aggregated DataFrame with single-level column headers
balance_agg.columns = balance_agg.columns.map('_'.join)  # Join the multi-level column names with an underscore
balance_agg.reset_index(drop=True, inplace=True)     # Reset the index and drop the old index

# Rename the 'ID_' column to 'ID'
balance_agg.rename(columns={'ID_': 'ID'}, inplace=True)

print(balance_agg.shape)
balance_agg.head()

(535943, 32)


,ID,month_count,balance_amount_len,balance_amount_sum,balance_amount_min,balance_amount_max,balance_amount_median,balance_amount_std,days_default_len,days_default_sum,...,entity_entity_2_sum,entity_entity_3_sum,entity_entity_4_sum,entity_entity_5_sum,type_type_1_sum,type_type_2_sum,type_type_3_sum,type_type_4_sum,type_type_5_sum,type_type_7_sum
0,00000f6dd2df6f081ce59cc4490f152e3fd5695de62bd9...,3,3,9.5710,3.1754,3.2202,3.1754,0.025865,3,0.000,...,0,0,3,0,0,0,3,0,0,0
1,000043cc1abd7597c99f494cc523ff99a4e0e64e98b128...,0,1,0.0000,NaN,NaN,NaN,NaN,1,0.000,...,0,0,0,0,0,0,0,0,0,0
2,00007a4554cd3a350aec583adebe766eb0beef5684f3dd...,60,60,203.6407,1.6760,4.1437,3.4855,0.641316,60,4.713,...,0,24,12,12,0,0,60,0,0,0
3,000095b7b77ff622b2130f2d4bfa1e8d81f1fc4580a248...,2,2,4.7712,2.3856,2.3856,2.3856,0.000000,2,10.000,...,0,0,0,2,0,0,2,0,0,0
4,0000a9728fc57abef25c2fc0424c9c907a5559e7b8b559...,37,37,150.1906,2.7285,4.9909,4.0671,0.784904,37,5.000,...,0,24,0,0,0,0,25,0,12,0


# Merge and Aggregate customers

In [25]:
# Merge universe_test with customers
customers_train = pd.merge(left = universe_train, right = customers, on=['ID'], how='left', suffixes=['_universe,', '_customers'])

In [49]:
# Dummify 'type_job' and 'bureau_risk' columns
customers_dummy = pd.get_dummies(customers_train, columns=['type_job', 'bureau_risk'], prefix=['type_job', 'bureau_risk'])

# Convert 'product_' and 'ofert_' columns from 'No' to 0 and 'Yes' to 1
customers_dummy.replace({'No': 0, 'Yes': 1}, inplace=True)

# Drop the 'attrition' column
customers_dummy.drop(columns=['attrition', 'period'], inplace=True)

print(balance_agg.shape)
customers_dummy.head()

(535943, 32)


,ID,age,income,time_from_specialized,payroll,product_1,product_2,product_3,product_4,ofert_1,...,type_job_type_7,bureau_risk_category_1,bureau_risk_category_2,bureau_risk_category_3,bureau_risk_category_4,bureau_risk_category_5,bureau_risk_category_6,bureau_risk_category_7,bureau_risk_category_8,bureau_risk_category_9
0,4d5a33701cb5b30b0b0a924d80de4ae78fbd0b54e2117f...,3.8498,4.6527,0.000,0.0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,bf7092e5f394d266143dfe90b3fc73eac51f0b0084d7a3...,4.5476,4.2447,4.274,0.0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,270d8f8f607d19886c50edb7746c3670194134b56d31e2...,4.3600,4.2117,0.000,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,5e6b7bea5e4911329669f45728d3398ad54dfe11fbb16d...,4.3914,4.1584,0.000,0.0,0,0,0,1,1,...,1,0,1,0,0,0,0,0,0,0
4,98954adf775b9fce1c9e311a025ec3e0a1c6e90f991ef7...,4.3077,4.3285,0.000,0.0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


# Merge and Aggregate digital

In [32]:
# Merge universe_test with digital
digital_train = pd.merge(left = universe_train, right = digital, on=['ID'], how='left', suffixes=['_universe', '_digital'])

In [34]:
# Drop the 'attrition' column
digital_train.drop(columns=['attrition', 'period_digital', 'period_universe'], inplace=True)

In [39]:
# Define aggregation functions for statistics
agg_functions = {
    'month': 'count',  # Length
    **{col: ['sum', 'min', 'max', np.median, np.std] for col in digital_train.columns if col.startswith('dig_')}
}

# Group by 'ID' and calculate the statistics
digital_agg = digital_train.groupby('ID').agg(agg_functions)

# Rename columns for clarity
digital_agg.columns = ['len_digital'] + [f'{col}_{stat}' for col in digital_train.columns if col.startswith('dig_') for stat in ['sum', 'min', 'max', 'median', 'std']]

# Reset the index to have 'ID' as a regular column
digital_agg.reset_index(inplace=True)

print(digital_agg.shape)
digital_agg.head()

(535943, 57)


,ID,len,dig_1_sum,dig_1_min,dig_1_max,dig_1_median,dig_1_std,dig_2_sum,dig_2_min,dig_2_max,...,dig_10_sum,dig_10_min,dig_10_max,dig_10_median,dig_10_std,dig_11_sum,dig_11_min,dig_11_max,dig_11_median,dig_11_std
0,00000f6dd2df6f081ce59cc4490f152e3fd5695de62bd9...,9,11.4030,0.0000,3.0905,1.3671,1.001256,29.3592,1.0457,4.2168,...,1.9388,0.0000,1.9388,0.00000,0.646267,20.0606,0.5703,3.9845,2.6324,1.281973
1,000043cc1abd7597c99f494cc523ff99a4e0e64e98b128...,0,0.0000,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,...,0.0000,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN
2,00007a4554cd3a350aec583adebe766eb0beef5684f3dd...,2,6.4158,1.4158,5.0000,3.2079,2.534412,8.7489,3.7489,5.0000,...,3.1621,1.2233,1.9388,1.58105,0.505935,1.5452,0.0001,1.5451,0.7726,1.092480
3,000095b7b77ff622b2130f2d4bfa1e8d81f1fc4580a248...,0,0.0000,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,...,0.0000,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN
4,0000a9728fc57abef25c2fc0424c9c907a5559e7b8b559...,11,31.5449,1.3679,5.0000,2.8281,1.143443,38.7035,2.6366,4.9355,...,7.7124,0.0000,3.2312,0.00000,1.099247,15.8620,0.0001,3.5520,1.2887,1.150386


# Merge and Aggregate liabilities

In [59]:
# Merge universe_test with liabilities
liabilities_train = pd.merge(left = universe_train, right = liabilities, on=['ID'], how='left', suffixes=['_universe', '_liabilities'])

In [61]:
# Drop the 'attrition' column
liabilities_train.drop(columns=['attrition', 'period_liabilities', 'period_universe'], inplace=True)

In [62]:
# Define aggregation functions for statistics
agg_functions = {
    'month': 'count',  # Length
    **{col: ['sum', 'min', 'max', np.median, np.std] for col in liabilities_train.columns if col.startswith('product_')}
}

# Group by 'ID' and calculate the statistics
liabilities_agg = liabilities_train.groupby('ID').agg(agg_functions)

# Rename columns for clarity
liabilities_agg.columns = ['len_liabilities'] + [f'{col}_{stat}' for col in liabilities_train.columns if col.startswith('product_') for stat in ['sum', 'min', 'max', 'median', 'std']]

# Reset the index to have 'ID' as a regular column
liabilities_agg.reset_index(inplace=True)

print(liabilities_agg.shape)
liabilities_agg.head()

(535943, 12)


,ID,len_liabilities,product_1_sum,product_1_min,product_1_max,product_1_median,product_1_std,product_2_sum,product_2_min,product_2_max,product_2_median,product_2_std
0,00000f6dd2df6f081ce59cc4490f152e3fd5695de62bd9...,12,16.8877,0.4939,3.4815,1.04545,1.089755,0.0000,0.000,0.0000,0.000,0.000000
1,000043cc1abd7597c99f494cc523ff99a4e0e64e98b128...,12,38.4301,2.3428,3.9980,3.31485,0.501052,12.9707,0.073,3.1542,1.263,1.058328
2,00007a4554cd3a350aec583adebe766eb0beef5684f3dd...,0,0.0000,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN
3,000095b7b77ff622b2130f2d4bfa1e8d81f1fc4580a248...,12,47.8380,3.9865,3.9865,3.98650,0.000000,0.0000,0.000,0.0000,0.000,0.000000
4,0000a9728fc57abef25c2fc0424c9c907a5559e7b8b559...,12,33.7255,2.0286,3.9150,2.88050,0.457376,0.0000,0.000,0.0000,0.000,0.000000


# Merge and Aggregate movements

In [45]:
# Merge universe_test with liabilities
movements_train = pd.merge(left = universe_train, right = movements, on=['ID'], how='left', suffixes=['_universe', '_movements'])

In [47]:
# Drop the 'attrition' column
movements_train.drop(columns=['attrition', 'period_movements', 'period_universe'], inplace=True)

In [50]:
# Define aggregation functions for statistics
agg_functions = {
    'month': 'count',  # Length
    **{col: ['sum', 'min', 'max', np.median, np.std] for col in movements_train.columns if col.startswith('type_')}
}

# Group by 'ID' and calculate the statistics
movements_agg = movements_train.groupby('ID').agg(agg_functions)

# Rename columns for clarity
movements_agg.columns = ['len_movements'] + [f'{col}_{stat}' for col in movements_train.columns if col.startswith('type_') for stat in ['sum', 'min', 'max', 'median', 'std']]

# Reset the index to have 'ID' as a regular column
movements_agg.reset_index(inplace=True)

print(movements_agg.shape)
movements_agg.head()

(535943, 22)


,ID,len_movements,type_1_sum,type_1_min,type_1_max,type_1_median,type_1_std,type_2_sum,type_2_min,type_2_max,...,type_3_sum,type_3_min,type_3_max,type_3_median,type_3_std,type_4_sum,type_4_min,type_4_max,type_4_median,type_4_std
0,00000f6dd2df6f081ce59cc4490f152e3fd5695de62bd9...,2,2.2460,0.0,2.2460,1.1230,1.588162,0.00,0.0,0.0000,...,0.0000,0.0,0.0000,0.00000,0.000000,2.6134,0.0,2.6134,1.3067,1.847953
1,000043cc1abd7597c99f494cc523ff99a4e0e64e98b128...,2,2.0298,0.0,2.0298,1.0149,1.435285,0.00,0.0,0.0000,...,3.4227,0.0,3.4227,1.71135,2.420214,0.0000,0.0,0.0000,0.0000,0.000000
2,00007a4554cd3a350aec583adebe766eb0beef5684f3dd...,6,2.9017,0.0,2.9017,0.0000,1.184614,0.00,0.0,0.0000,...,14.6370,0.0,3.4955,3.00670,1.308982,15.4955,0.0,3.3280,3.0021,1.284720
3,000095b7b77ff622b2130f2d4bfa1e8d81f1fc4580a248...,0,0.0000,NaN,NaN,NaN,NaN,0.00,NaN,NaN,...,0.0000,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN
4,0000a9728fc57abef25c2fc0424c9c907a5559e7b8b559...,5,2.8701,0.0,2.8701,0.0000,1.283548,5.79,0.0,2.9725,...,2.4001,0.0,2.4001,0.00000,1.073357,5.5136,0.0,2.8355,0.0000,1.510987


### Merge all databases

In [65]:
# Merge balance_agg and customers_dummy on 'ID'
merged_df = balance_agg.merge(customers_dummy, on='ID', how='left')

# Merge digital_agg on 'ID'
merged_df = merged_df.merge(digital_agg, on='ID', how='left')

# Merge liabilities_agg on 'ID'
merged_df = merged_df.merge(liabilities_agg, on='ID', how='left')

# Merge movements_agg on 'ID'
merged_df = merged_df.merge(movements_agg, on='ID', how='left')

# Merge universe_train on 'ID'
merged_df = merged_df.merge(universe_train, on='ID', how='left')

# Drop the 'period' column
merged_df.drop(columns=['period'], inplace=True)

print(merged_df.shape)
merged_df.isna().sum()

(535943, 148)


ID                         0
month_count                0
balance_amount_len         0
balance_amount_sum         0
balance_amount_min    203288
                       ...  
type_4_min            187092
type_4_max            187092
type_4_median         187092
type_4_std            229902
attrition                  0
Length: 148, dtype: int64

### NAs

In [ ]:
# Fill NaN values with 0 for all columns
merged_df.fillna(0, inplace=True)
merged_df.isna().sum().sum()
merged_df['attrition'] = merged_df['attrition'].astype('category')

0

In [89]:
# Filter columns starting with 'product_'
product_columns = [col for col in merged_df.columns if col.startswith('product')]

# Print the filtered columns
for col in product_columns:
    print(col)

product_product_1_sum
product_product_2_sum
product_product_3_sum
product_product_4_sum
product_product_5_sum
product_product_6_sum
product_product_7_sum
product_1
product_2
product_3
product_4
product_1_sum
product_1_min
product_1_max
product_1_median
product_1_std
product_2_sum
product_2_min
product_2_max
product_2_median
product_2_std


### Run Model

In [81]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Separate the features (X) and the target variable (y)
X = merged_df.drop(columns=['attrition', 'ID'])
y = merged_df['attrition']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Calculate the F1-score on the test data
f1_test = f1_score(y_test, y_pred, average='macro')

# Print the test F1-score
print("Test F1-score:", f1_test)



Test F1-score: 0.196008422986642


In [85]:
# Fit the model on the training data
rf_classifier.fit(X_train, y_train)

# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print the top N most important features
top_n = 10  # Change this value to see more or fewer top features
print(f"Top {top_n} Most Important Features:")
print(feature_importance_df.head(top_n))

Top 10 Most Important Features:
                   Feature  Importance
115          product_1_sum    0.034216
118       product_1_median    0.033337
116          product_1_min    0.032942
117          product_1_max    0.028270
119          product_1_std    0.028049
32                  income    0.026246
31                     age    0.024571
3       balance_amount_min    0.010811
4       balance_amount_max    0.010809
5    balance_amount_median    0.010783


In [83]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, train_test_split

# Separate the features (X) and the target variable (y)
X = merged_df.drop(columns=['attrition', 'ID'])
y = merged_df['attrition']

# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform cross-validation and calculate the F1-score
f1_scores = cross_val_score(rf_classifier, X, y, cv=5, scoring='f1_macro')

# Print the F1-scores for each fold
print("F1-scores for each fold:", f1_scores)

# Calculate and print the mean F1-score and standard deviation
mean_f1 = f1_scores.mean()
std_f1 = f1_scores.std()
print("Mean F1-score:", mean_f1)
print("Standard Deviation of F1-scores:", std_f1)


F1-scores for each fold: [0.19382694 0.18863559 0.19139828 0.19450784 0.1901017 ]
Mean F1-score: 0.1916940670641683
Standard Deviation of F1-scores: 0.0022110572216260975
